In [ ]:
%env WORKDIR=/tmp/vault

In [ ]:
import os
from dotenv import load_dotenv

load_dotenv("/tmp/vault/config.env")

VAULT_TOKEN = os.getenv('VAULT_TOKEN')
VAULT_ADDR = os.getenv('VAULT_ADDR')
VAULT_CACERT = os.getenv('VAULT_CACERT')

## Instala CertManager
https://cert-manager.io/docs/installation/kubectl/

In [ ]:
! kubectl apply -f https://github.com/cert-manager/cert-manager/releases/download/v1.19.1/cert-manager.yaml

## Verifica la instalación de los PODs

In [ ]:
! kubectl get pods --namespace cert-manager

In [ ]:
! cmctl check api --wait=2m

## Configura Vault como Issuer

### Modify ACL

In [ ]:
%%bash

vault policy write devk8s - <<EOF
path "kvv2/*" {
  capabilities = ["read", "list", "subscribe"]
  subscribe_event_types = ["*"]                     # https://developer.hashicorp.com/vault/docs/concepts/events#event-types
}
path "sys/events/subscribe/*" {
  capabilities = ["read"]
}
path "database/creds/readonly" {
  capabilities = [ "read"]
}
path "pki_int/issue/example-dot-com" {
  capabilities = [ "update" ]
}
path "pki_int/revoke" {
  capabilities = [ "update" ]
}
# Cert Manager needs sign permission to sign CSRs
path "pki_int/sign/example-dot-com" {
  capabilities = [ "update", "create" ]
}
EOF

### Add Role to Vault

In [ ]:
%%bash

vault write auth/kubernetes/role/issuer \
    bound_service_account_names=issuer \
    bound_service_account_namespaces=certmanager \
    policies=devk8s \
    ttl=10m

### Create Service Account

In [ ]:
%%bash
kubectl create namespace certmanager
kubectl create serviceaccount issuer -n certmanager
kubectl get sa -n certmanager

In [ ]:
%%bash
# Verify the certificate
kubectl create token issuer -n certmanager

### Create RBAC for CertManager to Create Tokens
https://cert-manager.io/docs/configuration/vault/#option-2-vault-authentication-method-use-kubernetes-auth

In [ ]:
%%bash
# Create a Role that allows creating tokens for the issuer service account
cat > $WORKDIR/rbac.yaml <<EOF
apiVersion: rbac.authorization.k8s.io/v1
kind: Role
metadata:
  name: cert-manager-serviceaccount-issuer
  namespace: certmanager
rules:
  - apiGroups: ['']
    resources: ['serviceaccounts/token']
    resourceNames: ['issuer']
    verbs: ['create']
---
apiVersion: rbac.authorization.k8s.io/v1
kind: RoleBinding
metadata:
  name: cert-manager-serviceaccount-issuer
  namespace: certmanager
roleRef:
  apiGroup: rbac.authorization.k8s.io
  kind: Role
  name: cert-manager-serviceaccount-issuer
subjects:
  - kind: ServiceAccount
    name: cert-manager
    namespace: cert-manager
EOF

kubectl apply -f $WORKDIR/rbac.yaml
kubectl get role,rolebinding -n certmanager

### Create CertManager Issuer

In [ ]:
%%bash
cat > $WORKDIR/issuer.yaml <<EOF
apiVersion: cert-manager.io/v1
kind: Issuer
metadata:
  name: vault-issuer
  namespace: certmanager
spec:
  vault:
    server: https://vault.vault.svc.cluster.local:8200
    # CertManager creates its own CSR and Private Key, that's why we use 'sign' endpoint
    path: pki_int/sign/example-dot-com 
    caBundleSecretRef:
      name: vault-ca
      key: ca.crt
    auth:
      kubernetes:
        mountPath: /v1/auth/kubernetes
        role: issuer
        serviceAccountRef:
          name: issuer
EOF

# Create a secret with Vault's CA certificate
kubectl create secret generic vault-ca \
  --from-file=ca.crt=$VAULT_CACERT \
  -n certmanager \
  --dry-run=client -o yaml | kubectl apply -f -

kubectl apply -f $WORKDIR/issuer.yaml
kubectl get issuer -n certmanager


In [ ]:
! kubectl describe issuer vault-issuer -n certmanager 

In [ ]:
! kubectl get issuer -n certmanager

## Request a certificate

In [ ]:
%%bash
cat > $WORKDIR/certificate.yaml <<EOF
apiVersion: cert-manager.io/v1
kind: Certificate
metadata:
  name: demo-example-com
  namespace: certmanager
spec:
  secretName: example-com-tls
  issuerRef:
    name: vault-issuer
  commonName: demo.example.com
  dnsNames:
  - demo.example.com
  - www.demo.example.com
  - www.example.com
  duration: 1h
EOF

kubectl apply -f $WORKDIR/certificate.yaml

### Verify certificate  

In [ ]:
%%bash
kubectl get certificate -n certmanager
kubectl describe certificate demo-example-com -n certmanager

In [ ]:
%%bash
sleep 10
kubectl get secrets example-com-tls -n certmanager
kubectl describe secrets example-com-tls -n certmanager

# Use Certificate and Reload in Rotation
* Cert-manager only manages the certificate Secret
* It does NOT automatically reload pods consuming them
* You need an external controller like Reloader

## Install Stakater Reloader

In [ ]:
! kubectl apply -f https://raw.githubusercontent.com/stakater/Reloader/master/deployments/kubernetes/reloader.yaml

In [ ]:
%%bash
sleep 20
kubectl get pods

In [ ]:
%%bash
cat > ${WORKDIR}/mypkiapp-certmanager.yaml <<'EOF'
apiVersion: v1
kind: ConfigMap
metadata:
  name: nginx-config
  namespace: certmanager
data:
  nginx.conf: |
    events {
        worker_connections 1024;
    }
    http {
        server {
            listen 443 ssl;
            server_name demo.example.com;
            
            ssl_certificate /etc/nginx/certs/tls.crt;
            ssl_certificate_key /etc/nginx/certs/tls.key;
            
            ssl_protocols TLSv1.2 TLSv1.3;
            ssl_ciphers HIGH:!aNULL:!MD5;
            
            location / {
                root /usr/share/nginx/html;
                index index.html;
            }
        }
    }
  index.html: |
    <!DOCTYPE html>
    <html>
    <head>
        <title>PKI Certificate Info</title>
        <style>
            body {
                font-family: Arial, sans-serif;
                margin: 0;
                padding: 20px;
                background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
                min-height: 100vh;
            }
            .container {
                max-width: 900px;
                margin: 0 auto;
                background: white;
                padding: 30px;
                border-radius: 10px;
                box-shadow: 0 10px 40px rgba(0,0,0,0.3);
            }
            h1 {
                color: #667eea;
                text-align: center;
                margin-bottom: 30px;
            }
            .cert-section {
                margin: 20px 0;
                padding: 20px;
                background: #f8f9fa;
                border-left: 4px solid #667eea;
                border-radius: 5px;
            }
            .cert-label {
                font-weight: bold;
                color: #555;
                margin-bottom: 5px;
            }
            .cert-value {
                color: #333;
                font-family: monospace;
                background: white;
                padding: 10px;
                border-radius: 3px;
                word-break: break-all;
            }
            .success {
                background: #d4edda;
                border-color: #28a745;
                color: #155724;
                padding: 15px;
                border-radius: 5px;
                margin: 20px 0;
            }
            .command {
                background: #2d3748;
                color: #68d391;
                padding: 15px;
                border-radius: 5px;
                margin: 20px 0;
                font-family: monospace;
                overflow-x: auto;
            }
        </style>
    </head>
    <body>
        <div class="container">
            <h1>Vault PKI Certificate Information</h1>
            <div class="success">
                This connection is secured with a certificate issued by HashiCorp Vault PKI via Cert-Manager!
            </div>
        </div>
    </body>
    </html>
---
apiVersion: apps/v1
kind: Deployment
metadata:
  name: mypkiapp-certmanager
  namespace: certmanager
  # Add the reloader annotations
  annotations:
    secret.reloader.stakater.com/auto: "true"
spec:
  replicas: 1
  selector:
    matchLabels:
      app: mypkiapp-certmanager
  template:
    metadata:
      labels:
        app: mypkiapp-certmanager
    spec:
      containers:
      - name: nginx
        image: nginx:latest
        ports:
        - containerPort: 443
          name: https
        volumeMounts:
        - name: certs
          mountPath: /etc/nginx/certs
          readOnly: true
        - name: nginx-config
          mountPath: /etc/nginx/nginx.conf
          subPath: nginx.conf
          readOnly: true
        - name: html
          mountPath: /usr/share/nginx/html/index.html
          subPath: index.html
          readOnly: true
      volumes:
      - name: certs
        secret:
          secretName: example-com-tls
          items:
          - key: tls.crt
            path: tls.crt
          - key: tls.key
            path: tls.key
      - name: nginx-config
        configMap:
          name: nginx-config
      - name: html
        configMap:
          name: nginx-config
---
apiVersion: v1
kind: Service
metadata:
  name: mypkiapp-certmanager
  namespace: certmanager
spec:
  selector:
    app: mypkiapp-certmanager
  ports:
  - port: 443
    targetPort: 443
    protocol: TCP
    name: https
  type: ClusterIP
EOF

kubectl apply -f ${WORKDIR}/mypkiapp-certmanager.yaml

In [ ]:
%%bash
# Verfiy the deployment and pods
kubectl get deployments -n certmanager
kubectl get pods -n certmanager

In [ ]:
%%bash
# Verify the new certificate and key match
echo "=== Verifying new certificate ==="
CERT_MOD=$(kubectl get secret example-com-tls -n certmanager -o jsonpath='{.data.tls\.crt}' | base64 -d | openssl x509 -noout -modulus | openssl md5)
KEY_MOD=$(kubectl get secret example-com-tls -n certmanager -o jsonpath='{.data.tls\.key}' | base64 -d | openssl rsa -noout -modulus 2>/dev/null | openssl md5)

echo "Certificate modulus: $CERT_MOD"
echo "Private key modulus:  $KEY_MOD"

if [ "$CERT_MOD" = "$KEY_MOD" ]; then
    echo ""
    echo "✅ SUCCESS! Certificate and private key match!"
else
    echo ""
    echo "❌ ERROR! Certificate and private key still don't match!"
fi

### Verificar que el certificado del POD es correcto

In [ ]:
%%bash
# From a terminal
echo "Testing HTTPS connection to the service:"
kubectl run -it --rm curl-test --image=curlimages/curl --restart=Never -n certmanager -- curl -k https://mypkiapp-certmanager.certmanager.svc.cluster.local | head -50

In [ ]:
%%bash
echo "Current certificate serial number:"
kubectl exec -n certmanager deploy/mypkiapp-certmanager -- openssl x509 -in /etc/nginx/certs/tls.crt -serial -noout
echo ""
echo "Current pods age:"
kubectl get pods -n certmanager -l app=mypkiapp-certmanager
echo ""
echo "Waiting 1hour for certificate renewal and rollout restart..."
echo ""


In [ ]:
%%bash
#sleep 3600
echo ""
echo "New certificate serial number (should be different):"
kubectl exec -n certmanager deploy/mypkiapp-certmanager -- openssl x509 -in /etc/nginx/certs/tls.crt -serial -noout
echo ""
echo "New pods (check age - should be recently restarted):"
kubectl get pods -n certmanager -l app=mypkiapp-certmanager